In [12]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
# Data process
import pandas as pd
import numpy as np
# SARIMA Model
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
# Other
import datetime
import pickle
from plotly_utils.utils import get_plotly_fig

%matplotlib inline
pylab.rcParams['figure.figsize'] = 12, 4

In [24]:
full_df = pd.read_pickle('../datasets/Full_Task2_Dataframe.pkl')
del full_df['EarlyPeakTrafficTime']
del full_df['LatePeakTrafficTime']
del full_df['NormalTrafficTime']

full_df.columns = [
    ['Volume', 'Volume', 'Volume', 'Volume', 'Volume',
        'Weather', 'Weather', 'Weather', 'Weather', 'Date', 'Date', 'Date', 'Date', 'Date', 'Date', 'Date'],
    ['T1D0', 'T1D1', 'T2D0', 'T3D0', 'T3D1', 'HeavyRain', 'LightRain',
     'Rain', 'Sunny', 'DayOfWeek', 'BeforeNationalDay',
     'NationalDayFront(1-4)', 'NationalDayTail(5-7)', 'Weekend',
     'WorkingDay', 'WorkingWeekend']]

In [27]:
full_df = full_df[['Volume','Date']]

In [47]:
full_df.columns = full_df.columns.set_levels(['Feature','Target','Feature'],level=0)

In [582]:
morning = full_df.between_time('15:00','19:00')

In [611]:
morning

Target                              Feature  \
                      T1D0   T1D1  T2D0   T3D0   T3D1 DayOfWeek   
2016-09-19 15:00:00   48.0   77.0  77.0  117.0   75.0         1   
2016-09-19 15:20:00   57.0  104.0  67.0  109.0   95.0         1   
2016-09-19 15:40:00   43.0   92.0  74.0  115.0   83.0         1   
2016-09-19 16:00:00   42.0  110.0  76.0  123.0  101.0         1   
2016-09-19 16:20:00   46.0  122.0  86.0  111.0   90.0         1   
2016-09-19 16:40:00   55.0  116.0  99.0  116.0  115.0         1   
2016-09-19 17:00:00   36.0   96.0  85.0  110.0   86.0         1   
2016-09-19 17:20:00   34.0  116.0  72.0  118.0   85.0         1   
2016-09-19 17:40:00   24.0  112.0  53.0   95.0   91.0         1   
2016-09-19 18:00:00   29.0   67.0  55.0   94.0   73.0         1   
2016-09-19 18:20:00   27.0   94.0  50.0   56.0   78.0         1   
2016-09-19 18:40:00   18.0   65.0  49.0   61.0   77.0         1   
2016-09-19 19:00:00   13.0   62.0  44.0   51.0   44.0         1   
2016-09-20 15:00:00   71.0   78.0  62.0  124.0   80.0         2   
2016-09-20 15:20:00   64.0   79.0  69.0  132.0   73.0         2   
2016-09-20 15:40:00   73.0   71.0  69.0  131.0   87.0         2   
2016-09-20 16:00:00   48.0   95.0  82.0  122.0   85.0         2   
2016-09-20 16:20:00   57.0   96.0  76.0  113.0   97.0         2   
2016-09-20 16:40:00   48.0   95.0  64.0  122.0   96.0         2   
2016-09-20 17:00:00   38.0  111.0  82.0  126.0   84.0         2   
2016-09-20 17:20:00   42.0  110.0  86.0  116.0   86.0         2   
2016-09-20 17:40:00   38.0  133.0  85.0   95.0  106.0         2   
2016-09-20 18:00:00   26.0   78.0  77.0   81.0   90.0         2   
2016-09-20 18:20:00   29.0   60.0  60.0  100.0   84.0         2   
2016-09-20 18:40:00   20.0   53.0  56.0   63.0   88.0         2   
2016-09-20 19:00:00   17.0   44.0  61.0   59.0   53.0         2   
2016-09-21 15:00:00   48.0   98.0  63.0  113.0   98.0         3   
2016-09-21 15:20:00   46.0   96.0  81.0  114.0  108.0         3   
2016-09-21 15:40:00   49.0   90.0  77.0   89.0   99.0         3   
2016-09-21 16:00:00   46.0  105.0  66.0  109.0   87.0         3   
...                    ...    ...   ...    ...    ...       ...   
2016-10-22 18:00:00    NaN    NaN   NaN    NaN    NaN         6   
2016-10-22 18:20:00    NaN    NaN   NaN    NaN    NaN         6   
2016-10-22 18:40:00    NaN    NaN   NaN    NaN    NaN         6   
2016-10-22 19:00:00    NaN    NaN   NaN    NaN    NaN         6   
2016-10-23 15:00:00   61.0  101.0  76.0  122.0  137.0         7   
2016-10-23 15:20:00   65.0  124.0  83.0  109.0  117.0         7   
2016-10-23 15:40:00   61.0  114.0  74.0  133.0   89.0         7   
2016-10-23 16:00:00   60.0  104.0  61.0  122.0   95.0         7   
2016-10-23 16:20:00   58.0  113.0  72.0  110.0   91.0         7   
2016-10-23 16:40:00   62.0  108.0  61.0  122.0  104.0         7   
2016-10-23 17:00:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 17:20:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 17:40:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 18:00:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 18:20:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 18:40:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-23 19:00:00    NaN    NaN   NaN    NaN    NaN         7   
2016-10-24 15:00:00   50.0   82.0  67.0  141.0   80.0         1   
2016-10-24 15:20:00   47.0   80.0  73.0  105.0  102.0         1   
2016-10-24 15:40:00   26.0  103.0  71.0  105.0   71.0         1   
2016-10-24 16:00:00   49.0   99.0  69.0  115.0  101.0         1   
2016-10-24 16:20:00   50.0  108.0  75.0  100.0   80.0         1   
2016-10-24 16:40:00   44.0  103.0  80.0  130.0   81.0         1   
2016-10-24 17:00:00    NaN    NaN   NaN    NaN    NaN         1   
2016-10-24 17:20:00    NaN    NaN   NaN    NaN    NaN         1   
2016-10-24 17:40:00    NaN    NaN   NaN    NaN    NaN         1   
2016-10-24 18:00:00    NaN    NaN   NaN    NaN    NaN         1   


In [420]:
morning_target = morning_t.iloc[:,-30:]
morning_feature = morning_t.iloc[:,:30]
holiday = full_task2_df.groupby(full_task2_df.index.map(lambda dt:dt.date())).first()
morning_feature.columns = [col[1]+"_"+str(col[0])[:-3] for col in morning_feature.columns.values]
# morning_feature = np.log(morning_feature+1)
morning_feature = morning_feature.join(holiday)
morning_target.columns = [col[1]+"_"+str(col[0])[:-3] for col in morning_target.columns.values]

In [263]:
full_task2_df = pd.read_pickle("../datasets/full_task2_df.pkl")

In [264]:
full_task2_df = full_task2_df.iloc[:,-8:]

In [357]:
row = ['2016-09-19', '2016-09-20', '2016-09-21', '2016-09-22',
               '2016-09-23', '2016-09-24', '2016-09-25', '2016-09-26',
               '2016-09-27', '2016-09-28', '2016-09-29','2016-10-09', '2016-10-10', '2016-10-11', '2016-10-12',
               '2016-10-13', '2016-10-14', '2016-10-15', '2016-10-16',
               '2016-10-17', '2016-10-18', '2016-10-19', '2016-10-20',
               '2016-10-21', '2016-10-22', '2016-10-23', '2016-10-24']

In [421]:
morning_feature_no= morning_feature.loc[pd.to_datetime(row)]
morning_target_no = morning_target.loc[pd.to_datetime(row)]

In [588]:
split = 17
train = morning_feature_no.iloc[:split, :], morning_target_no.iloc[:split, :]
test = morning_feature_no.iloc[split:split+3, :], morning_target_no.iloc[split:split+3, :]

In [606]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=8, n_estimators=300,bootstrap=True)
model.fit(train[0], train[1])
np.mean(np.abs(model.predict(test[0].values) -
               test[1].values) / test[1].values)

0.16841067435977669

In [610]:
model.predict(test[0]).astype(int).astype(float)

array([[  66.,  106.,   87.,  137.,  122.,   59.,  111.,   94.,  136.,
         128.,   53.,  110.,   91.,  130.,  123.,   57.,  108.,   91.,
         124.,  122.,   59.,  101.,   79.,  123.,  104.,   53.,   88.,
          83.,  112.,   89.],
       [  61.,  104.,   85.,  130.,  115.,   58.,  110.,   95.,  133.,
         123.,   51.,  106.,   92.,  122.,  120.,   54.,  105.,   93.,
         120.,  121.,   58.,   93.,   82.,  117.,  106.,   53.,   88.,
          85.,  113.,   89.],
       [  55.,  121.,  114.,  159.,  159.,   48.,  119.,  112.,  154.,
         163.,   49.,  130.,  104.,  150.,  148.,   56.,  127.,   97.,
         141.,  146.,   51.,  127.,   79.,  132.,  122.,   50.,   92.,
          81.,  114.,   89.]])

In [608]:
test[1].values

array([[  47.,  103.,  118.,   97.,   80.,   55.,  123.,  103.,  128.,
         121.,   70.,  110.,  108.,  127.,   90.,   72.,  103.,  124.,
         115.,   97.,   55.,  121.,   95.,  121.,  105.,   60.,  114.,
         111.,  119.,   97.],
       [  44.,   72.,   92.,  114.,   80.,   35.,   67.,   99.,  102.,
          86.,   51.,  100.,   95.,  105.,  107.,   47.,  108.,  107.,
          94.,   90.,   48.,  108.,   87.,   72.,   98.,   38.,   88.,
          90.,  102.,  107.],
       [  46.,  140.,  133.,  182.,  147.,   56.,  141.,  109.,  180.,
         150.,   43.,  161.,  114.,  138.,  136.,   54.,  120.,  102.,
         150.,  137.,   41.,  121.,   91.,  117.,  106.,   46.,  101.,
          65.,  127.,   78.]])

In [511]:
np.abs(model.predict(test[0].values.reshape(
        1, -1)) - test[1].values) 

array([[  8.6,  28.4,  19.8,  37.2,  29.8,   5.4,  28.8,   5.6,  20. ,
         11.2,   1.8,  34.2,   8.6,  13. ,  16.6,   4.8,   7.8,   8.8,
         15.8,  20. ,   5.8,  12. ,  13.4,   0.2,   8.8,   1.2,   3.8,
          8. ,  27.2,   7.2]])